In [2]:
#using Qwen generate summary

from datasets import load_dataset

import pandas as pd
import re
from tqdm import tqdm
import numpy as np
import json
# Set OpenAI's API key and API base to use vLLM's API server.
from scipy.stats import kendalltau


prompt = "You are a helpful summary assistant. You can help users summarize news in two sentences."


from openai import OpenAI          
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8880/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

#load dataset
from datasets import load_dataset

#/home/xbr/LLM/dataset/summary/sample_dataset
#cnndm_sample_0k5_1k5.json  newsroom_sample_0k5_1k5.json  xsum_sample_0k5_1k5.json

In [7]:
data = pd.read_json('/home/xbr/LLM/benchmark_llm_summarization/pairwise_evaluation_results.json')
data = data.to_dict(orient='records')

# news_list = data['text']

# summary_list = []
   
for i in tqdm(range(len(data))):
    news = data[i]['article_text']
    
    chat_response = client.chat.completions.create(
        model="Qwen/Qwen1.5-72B-Chat",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": "Summarize the news in two sentences.\nArticle: "+news+'\nSummary: '},
            
        ],
        temperature=0
    )
    
    res = chat_response.choices[0].message.content
    
    data[i]['qwen_summary'] = res
    


import json   
with open('/home/xbr/LLM/benchmark_llm_summarization/pair_with_qwen.json', 'w') as f:
    json.dump(data, f, indent=4)
print("done")

  0%|          | 0/599 [00:00<?, ?it/s]

100%|██████████| 599/599 [49:18<00:00,  4.94s/it]

done


In [3]:
#generate qwen summary
data = json.load(open('/home/xbr/LLM/benchmark_llm_summarization/likert_evaluation_results_cnndm_average.json'))

for i in tqdm(range(len(data))):
    news = data[i]['article_text']
    
    chat_response = client.chat.completions.create(
        model="Qwen/Qwen1.5-72B-Chat",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": "Summarize the news in two sentences.\nArticle: "+news+'\nSummary: '},
            
        ],
        temperature=0
    )
    
    res = chat_response.choices[0].message.content
    
    data[i]['qwen_summary'] = res
    
#save to json 
with open('/home/xbr/LLM/benchmark_llm_summarization/likert_evaluation_results_cnndm_average_with_qwen.json', 'w') as f:
    json.dump(data, f, indent=4)

dict_keys(['article', 'summary', 'model', 'faithfulness', 'coherence', 'relevance'])
